In [1]:
# https://www.kaggle.com/c/facial-keypoints-detection/data

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import seaborn as sns
import keras as K
from VGG16 import VGG16_Obj

import tensorflow as tf
import os 
import sys
from datetime import datetime

from TrainImageObj import TrainImage # be careful with the tab vs ' '
# object used to debug and evaluate imported images
#CAREFUL: I did not handle nans, and in the presence of a nan an error occurs 

cwd = os.getcwd()
test = False
start = datetime.now()

projectDirectory = "C:\\Users\\Ted\\Projects\\Python\\FacialKeypointsDetection"
dataDir = "E:\\TK_PracticeDatabases\\facial-keypoints-detection"
os.chdir(dataDir)
testData = pd.read_csv("test.csv")
trainingData = pd.read_csv("training.csv")
idLookupData = pd.read_csv("IdLookupTable.csv")
sampleSubData = pd.read_csv("SampleSubmission.csv")

os.chdir(cwd)

if test:
    print(trainingData)

importComplete = datetime.now()
print("Import Finished: " + str(importComplete - start))

C:\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Import Finished: 0:00:02.957581


In [2]:
start = datetime.now()

os.chdir(projectDirectory)
vgg16 = VGG16_Obj()

importComplete = datetime.now()
print("Vgg16 Generated: " + str(importComplete - start))

C:\Users\Ted\Projects\Python\FacialKeypointsDetection\VGG16.py:51: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="block1_conv1", padding="same")`
  name='block1_conv1')
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\VGG16.py:144: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(512, (3, 3), activation="relu", name="block5_conv2", padding="same")`
  name='block5_conv2')


Vgg16 Generated: 0:00:02.319057


In [5]:
model = vgg16.modelNoFC
model.summary() # should say 14,714,688 untrainable parameters


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [18]:
def weighted_binary_crossentropy(y_true, y_pred):
    # Oiginal binary crossentropy (see losses.py):
    # K.mean(K.binary_crossentropy(y_true, y_pred), axis=-1)
    # Calculate the binary crossentropy
    
    one_weight = 0.75
    zero_weight = 1.0 - one_weight
    b_ce = K.binary_crossentropy(y_true, y_pred)
    # Apply the weights
    weight_vector = y_true * one_weight + (1. - y_true) * zero_weight
    weighted_b_ce = weight_vector * b_ce
    # Return the mean error
    return K.mean(weighted_b_ce)

# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - VGG16 sequentially network
    - boolean on training CNN layers
    - boolean for using the CNN layers as feature extractor
    - boolean on training Neural Net layers
    - weight to the 1s 
        - if =np.nan then weights are .5 and .5 for weighted X-entropy
    - Output: full architecture
- Second Function
    - Training Layer
    - Output: Fully Trained Model
- Third Function
    - Testing Layer
    - Output: Model to be used on Data
    
# Defining Arcitecture for Neural Net
## OBJ
- __init__
    - defines functions that can be called only 
- First Function
    - K fold cross validation
    - input 2 ints 
        - number of 0s
        - number of 1s
            - if only the first one populated then force weighted k fold to be false
    - number of bins
        - if bins = np.nan, then calculate even number of bins, if %bins  less than bins then add to final  
    - boolean for weighted k fold (evenly distributed) making even 1s in each bin
        - just incase unweighted 0s and 1s
        - separate the input vector into 2
    - Output: bins that are itterable


In [2]:
# save all images that are not labeled and labeled until a nan appears 
# for the first time 

imageDir = "SubplotImages"
os.chdir(imageDir)
for i in range(0,209):#,len(trainingData)): # only need so many (Nans make anything beyond this tricky to say the least)
    patient = TrainImage(trainingData, i) # load the images, labeled and not labeled
    plt.subplot(121)
    plt.imshow(patient.image)
    plt.title(str(i)+" Base")
    plt.subplot(122)
    plt.imshow(patient.labeledImage)
    plt.title(str(i)+" Labeled")
    plt.savefig(str(i)+" Base&Labeled")
    plt.close()
os.chdir(cwd)

C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_y"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_center_x"][index] = 0
C:\Users\Ted\Projects\Python\FacialKeypointsDetection\TrainImageObj.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  trainData["left_eye_inner_corner_y"][index]

KeyboardInterrupt: 